# Grid World --- dual agent coverage
##### Charles Zhang
##### Jul 14

###  3x4 Grid World

In [1]:
import numpy as np
import datetime

In [2]:
BOARD_ROWS = 3
BOARD_COLS = 4
START = (0, 0)
END = (0, 0)

In [3]:
class State:

    def __init__(self, state=(START, START)):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.state = state    # tuple of the coordinate
        self.is_end = False

    def check_end(self):
        if self.state == END:
            self.is_end = True

    def next_position(self, action):
        if action == "up":
            next_state = (self.state[0] - 1, self.state[1])
        elif action == "down":
            next_state = (self.state[0] + 1, self.state[1])
        elif action == "left":
            next_state = (self.state[0], self.state[1] - 1)
        else:
            next_state = (self.state[0], self.state[1] + 1)
        # boundary condition 
        if (next_state[0] >= 0) and (next_state[0] < BOARD_ROWS):
            if (next_state[1] >= 0) and (next_state[1] < BOARD_COLS):
                return next_state
        return self.state

In [4]:
class Agent:

    def __init__(self):
        self.actions = ["up", "down", "left", "right"]      # space
        self.State = State()
        self.is_end = self.State.is_end
        self.alpha = 0.1      # learning rate
        self.exp_rate = 0.95     # epsilon-greedy parameter
        self.decay_gamma = 0.9
        self.Q1_values = {}             # init Q1 table (dict)
        for i in range(BOARD_ROWS):    
            for j in range(BOARD_COLS):
                self.Q1_values[(i, j)] = {}
                for a in self.actions:
                    self.Q1_values[(i, j)][a] = 0
        
        self.Q2_values = {}             # init Q2 table (dict)
        for i in range(BOARD_ROWS):    
            for j in range(BOARD_COLS):
                self.Q2_values[(i, j)] = {}
                for a in self.actions:
                    self.Q2_values[(i, j)][a] = 0
                    
        # init a list to check if each grid is past          
        self.past_all = np.zeros((BOARD_ROWS, BOARD_COLS), dtype=bool).tolist()
        self.steps = []         # store the steps for each episode 
    
    def optimal_action(self):
        # greedy move
        max_value = -10000
        action = ""
        for a in self.actions:
            next_value = self.Q_values[self.State.state][a]
            if next_value >= max_value:
                action = a
                max_value = next_value
        return action
    
    def get_action(self):
        """
        The agent should choose randomly among the positions that have
        not been visited, and if all possible positions are visited, 
        then move randomly and receive a negative reward
        """
        state1, state2 = self.State.state[0], self.State.state[1]
        if np.random.uniform(0, 1) <= self.exp_rate:
            t = []
            for a in self.actions:
                p = self.State.next_position(a)
                if self.past_all[p[0]][p[1]] is True:
                    t.append(1)
                else:
                    t.append(0)
            if sum(t) == len(t):
                return np.random.choice(self.actions)
            # Else try to get an available unvisited position randomly
            else:
                while True:
                    action = np.random.choice(self.actions)
                    next_state = self.State.next_position(action)
                    if self.past_all[next_state[0]][next_state[1]] is False:
                        return action
                    else:
                        continue
        else:
            return self.optimal_action()
    
    def give_reward(self):
        """
        + 1 reward for visiting the unvisited grid
        -.1 reward for visiting the visited grid
        """
        reward = 0
#         if self.State.state==END:
#             reward += 1
        if self.past_all[self.State.state[0]][self.State.state[1]] is False:
            reward += 1
        else:
            reward -= .1
        return reward
    
    def set_action(self, action1, action2):
        state1, state2 = self.State.state[0], self.State.state[1]
        
        self.past_all[state1[0]][state1[1]] = True
        self.past_all[state2[0]][state2[1]] = True
        
        # get the next state
        next_state1 = self.State.next_position(action1)
        next_state2 = self.State.next_position(action2)  
        self.State = State(state=(next_state1, next_state2))
        
        reward = self.give_reward()
        # set the next state past
        if self.past_all[next_state1[0]][next_state1[1]] is False:
            self.past_all[next_state1[0]][next_state1[1]] = True
        
        if self.past_all[next_state2[0]][next_state2[1]] is False:
            self.past_all[next_state2[0]][next_state2[1]] = True
            
        # give the global reward if finish an epsiode
        if self.State.state==END and self.check_all_past():
            reward += 1
        # update the Q table
        qs_of_next_state = []
        for q_value in self.Q1_values[state1]:  
            qs_of_next_state.append(self.Q1_values[state1][q_value])
        delta = self.alpha*(reward + self.decay_gamma*(max(qs_of_next_state)) -
                            self.Q1_values[state1][action1])
        self.Q1_values[state1][action1] = round(self.Q1_values[state1][action1]+delta, 4)
        
        qs_of_next_state = []
        for q2_value in self.Q2_values[state2]:  
            qs_of_next_state.append(self.Q2_values[state2][q_value])
        delta = self.alpha*(reward + self.decay_gamma*(max(qs_of_next_state)) -
                            self.Q2_values[state2][action2])
        self.Q2_values[state2][action2] = round(self.Q2_values[state2][action2]+delta, 4)

    def reset(self):
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.past_all[i][j] = False
        self.State = State()
        self.is_end = self.State.is_end
    
    def check_all_past(self):
        for i in self.past_all:
            for j in i:
                if j is False:
                    return False
        return True
    
    def show_path(self):
        for i in range(BOARD_ROWS):
            print('---------------------------------------')
            row_string = "| "
            for j in range(BOARD_COLS):
                best_move = ""
                best_val = -1000
                for a in self.Q_values[(i,j)]:
                    if self.Q_values[(i,j)][a] > best_val:
                        best_val = self.Q_values[(i,j)][a]
                        best_move = a            
                row_string = row_string + " " + best_move + " |"
            print(row_string)
        print('---------------------------------------')
        
    def show_convergence(self, rounds=1000):
        import matplotlib.pyplot as plt
        %matplotlib inline
        x = []
        for i in range(rounds):
            x.append(i)
        y = self.steps
        plt.plot(x,y)
        plt.ylabel('Number of Steps')
        plt.xlabel('Episode')
        plt.title('Q Learning Convergence')
        plt.show()

    def train(self, rounds=1000):
        print("Training...")
        for r in range(rounds):
            t = str(round(r/(rounds/10)*10)) + '%|'
            for i in range(int(r/(rounds/10))):
                t += '='
            for i in range(int(10-r/(rounds/10))+1):
                t += ' '
            print(t+'|100%', end="\r")          
            self.reset()
            self.exp_rate *= 0.99  # decaying epsilon-greedy
            #self.alpha *= 1.01
            
            step = 0
            while True:
                action1 = self.get_action()
                action2 = self.get_action()
                self.set_action(action1, action2)
                self.State.check_end()
                self.is_end = self.State.is_end
                step += 1
#                 if self.is_end:
#                     break
                if self.is_end and self.check_all_past():
                    break
                if step > 5000:
                    break
            self.steps.append(step)
        print('100%|==========|100%')
        print("Training finished!")

In [5]:
agent = Agent()
start_time = datetime.datetime.now()
agent.train(rounds=650)
end_time = datetime.datetime.now()
time_period = (end_time - start_time).total_seconds()
print(agent.Q1_values)
print(agent.Q2_values)

Training...


TypeError: unsupported operand type(s) for -: 'tuple' and 'int'

In [ ]:
print("Running time: ", time_period)

In [ ]:
agent.show_convergence(rounds=650)

In [ ]:
min(agent.steps)

#### Optimal Policy

In [ ]:
agent.show_path()